In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from sklearn.ensemble import IsolationForest 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
import numpy as np
from xgboost import XGBRegressor
from boruta import BorutaPy

In [17]:
acs = pd.read_csv(r'C:\Users\IknaShillingford\Downloads\acs5yr2010_cook.csv') 
mapping = pd.read_excel('C:\\Users\\IknaShillingford\\Downloads\\il17trf.xlsx', sheet_name='il17trf')
energy = pd.read_csv(r'C:\Users\IknaShillingford\Downloads\energy_chicago.csv')

In [18]:
acs['GEO_ID'] = acs.GEO_ID.str.rsplit('US',expand=True)[1]
acs = pd.merge(acs, mapping[['GEOID10','AREALANDPT']], how='left', left_on= acs['GEO_ID'], right_on= mapping['GEOID10'].astype('str'))
acs['GEOID10'].isna().sum()

energy['GEO_ID_TRACT'] = energy['CENSUS BLOCK'].astype('str').str[:11]
e_tot = energy.groupby(['COMMUNITY AREA NAME','GEO_ID_TRACT'])[['TOTAL KWH','TOTAL THERMS']].sum()

df = acs.merge(e_tot, how='right', right_on='GEO_ID_TRACT', left_on='GEO_ID')
df['GEOID10'].isna().sum()
df.drop(columns=['key_0','Unnamed: 0'], inplace=True)
df.to_csv('acs_energy.csv')
df = df.dropna()


y1 = df['TOTAL KWH']
y2 = df['TOTAL THERMS']
drop_cols = [0,1,2,3,4,-1,-2,-4]
X = df.drop(df.columns[drop_cols], axis=1)

In [19]:
missing_values = X.isna().sum()
columns_with_missing_values = missing_values[missing_values > 0]

# Print the columns with missing values along with the number of missing values
print("Columns with missing values:")
print(columns_with_missing_values)

params = dict()
params["device"] = "cuda"
np.int = np.int32
np.float = np.float64
np.bool = np.bool_


X.reset_index(drop=True, inplace=True)

# Check for missing values
print("\nMissing Values:")
print(X.isnull().sum())

# Get summary information about the DataFrame
print("\nDataFrame Info:")
print(X.info())


Columns with missing values:
Series([], dtype: int64)

Missing Values:
Estimate!!BEDROOMS!!1 bedroom                          0
Estimate!!BEDROOMS!!2 bedrooms                         0
Estimate!!BEDROOMS!!3 bedrooms                         0
Estimate!!BEDROOMS!!4 bedrooms                         0
Estimate!!BEDROOMS!!5 or more bedrooms                 0
                                                      ..
Estimate!!YEAR STRUCTURE BUILT!!Built 1990 to 1999     0
Estimate!!YEAR STRUCTURE BUILT!!Built 2000 to 2004     0
Estimate!!YEAR STRUCTURE BUILT!!Built 2005 or later    0
Estimate!!YEAR STRUCTURE BUILT!!Total housing units    0
AREALANDPT                                             0
Length: 218, dtype: int64

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Columns: 218 entries, Estimate!!BEDROOMS!!1 bedroom to AREALANDPT
dtypes: float64(19), int64(199)
memory usage: 1.7 MB
None


In [ ]:
X.reset_index(drop=True, inplace=True)
y1.reset_index(drop=True, inplace=True)
y2.reset_index(drop=True, inplace=True)

# Replace the '!' characters in column names with underscores
X.columns = X.columns.str.replace('!', '_')
X.columns = X.columns.str.replace('Estimate__', '')
#X.columns = X.columns.str.replace(' ', '')
# Check the modified column names
print(X.columns)

X.reset_index(drop=True, inplace=True)


#Feature selection based on electricity consumption as target variable
xgb = XGBRegressor(n_estimators=100, random_state=6, params=params)
boruta_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=6, perc=85)
boruta_selector.fit(X.to_numpy(), y2)

In [ ]:
#Output feature selection results
feature_names = X.columns
feature_ranks = list(zip(feature_names, boruta_selector.ranking_, boruta_selector.support_, boruta_selector.support_weak_ )) 
boruta_results_1 = pd.DataFrame(feature_ranks, columns=['feature_name','rank', 'confirmed','tentative'])

#Feature selection based on electricity consumption as target variable
xgb = XGBRegressor(n_estimators=100, random_state=6, params=params)
boruta_selector = BorutaPy(xgb, n_estimators='auto', verbose=2, random_state=6, perc=85)
boruta_selector.fit(X.to_numpy(), y1)

#output feature selection results (electricity as target variable)
feature_names = X.columns
feature_ranks = list(zip(feature_names, boruta_selector.ranking_, boruta_selector.support_, boruta_selector.support_weak_ )) 
boruta_results_2 = pd.DataFrame(feature_ranks, columns=['feature_name','rank', 'confirmed','tentative'])

In [ ]:
# Get the features to keep based on Boruta results
features_to_keep = boruta_results_1[(boruta_results_1['confirmed'] == True) | (boruta_results_1['tentative'] == True)]['feature_name'].tolist()
# Filter the DataFrame to keep only the selected features
df_filtered = X[features_to_keep]

# Now 'df_filtered' contains only the features selected by Boruta (confirmed or tentative)
df_filtered['TOTAL KWH'] = y1 
#df_filtered['TOTAL THERMS'] = y2 


In [ ]:
df_filtered.to_csv('output.csv', index=False)

In [48]:
from sklearn.preprocessing import StandardScaler

In [65]:
# Separate features and target variable
Xs = df_filtered.drop(['TOTAL KWH'], axis=1)  # Assuming 'TOTAL KWH' is the target variable
ys = df_filtered['TOTAL KWH']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)

# Define a dictionary to store MSE for each model
mse_dict = {}

In [66]:
# Train and evaluate Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)
mse_lr = mean_squared_error(y_test, y_pred_lr)
mse_dict['Linear Regression'] = mse_lr

In [68]:
# Train and evaluate Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
mse_dict['Ridge Regression'] = mse_ridge


In [69]:

# Train and evaluate Lasso Regression
lasso_model = Lasso(alpha=1.0)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
mse_dict['Lasso Regression'] = mse_lasso

c:\Users\IknaShillingford\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.122e+16, tolerance: 8.866e+12
  model = cd_fast.enet_coordinate_descent(


In [70]:
# Train and evaluate Decision Tree
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
mse_dict['Decision Tree'] = mse_tree


In [71]:
# Train and evaluate Random Forest
forest_model = RandomForestRegressor(n_estimators=100, random_state=42)
forest_model.fit(X_train, y_train)
y_pred_forest = forest_model.predict(X_test)
mse_forest = mean_squared_error(y_test, y_pred_forest)
mse_dict['Random Forest'] = mse_forest

In [72]:
# Train and evaluate Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)
mse_gb = mean_squared_error(y_test, y_pred_gb)
mse_dict['Gradient Boosting'] = mse_gb


In [73]:
# Train and evaluate XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mse_dict['XGBoost'] = mse_xgb

In [74]:
best_model = min(mse_dict, key=mse_dict.get)
best_mse = mse_dict[best_model]
print(f'Best Model: {best_model}')
print(f'Best Mean Squared Error: {best_mse}')


Best Model: XGBoost
Best Mean Squared Error: 21817268597751.01


In [75]:
from sklearn.model_selection import GridSearchCV

# Define XGBoost regressor model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# Define hyperparameters for grid search
param_grid = {
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Perform grid search for hyperparameter tuning
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best estimator from grid search
best_xgb_model = grid_search.best_estimator_

# Predict energy usage for test data using the best model
y_pred_best_xgb = best_xgb_model.predict(X_test)

# Evaluate the best model
mse_best_xgb = mean_squared_error(y_test, y_pred_best_xgb)
print('Mean Squared Error (Best XGBoost after Hyperparameter Tuning):', mse_best_xgb)

# Print the best hyperparameters
print('Best Hyperparameters:', grid_search.best_params_)


Mean Squared Error (Best XGBoost after Hyperparameter Tuning): 18641497495918.99
Best Hyperparameters: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}


In [ ]:
# Save the best model to a file
import joblib
joblib.dump(best_xgb_model, 'best_xgb_model.pkl')

# Load the model from the file
loaded_model = joblib.load('best_xgb_model.pkl')

# Use the loaded model to make predictions
#predictions = loaded_model.predict(new_data)